# 1. Import libraries

In [ ]:
# data manipulation
import numpy as np
import pandas as pd
import re
from datetime import datetime
import time

# data collection
import requests
from bs4 import BeautifulSoup

# progress display
from tqdm import tqdm

# 2. Data collection and preparation

### Articles

In [ ]:
# Create Dataset
dataset = pd.DataFrame(columns=['Date', 'Title', 'Content'])

### Link searching ###
# data from the last 5 years due to a rather large dataset
history = [date.strftime("%Y-%m-%d") for date in pd.date_range(start="2008-01-01", end=datetime.today())]
for date in tqdm(history):
    # print(date)
    url = 'https://forsal.pl/archiwum/'+str(date)
#     print(url)
    f = requests.get(url)
    soup = BeautifulSoup(f.text, 'html.parser')
    mat = str(soup)
    links = (re.findall(r"<a.*? href=\"(.*?)\".*?>(.*?)</a>", mat)[:])
    for link in links:
        if (('artykuly' in link[0])&('https://www.gazetaprawna.pl/artykuly' not in link[0])&('swiat' not in link[0]))&\
            (('notowania' in link[0])|('finanse/gielda' in link[0])):
            # print(link[0])
            url2 = link[0]
            time.sleep(0.001)
            f = requests.get(url2)
            soup = BeautifulSoup(f.text, 'html.parser')
            title = link[1]
            # print(title)
            # print(date)
            try:
                content = soup.find_all(class_='leadDiv')[0].text[:]+' '+soup.find_all(class_='articleBody')[0].text.split('>>>')[0]
            except:
                try:
                    content = soup.find_all(class_='lead')[0].text[:]+' '+soup.find_all(class_='articleBody')[0].text.split('>>>')[0]
                except:
                    continue
            # print(content)
            # print()
            dataset_single_article = pd.DataFrame(({'Date':[date], 'Title':title, 'Content':content}))
            dataset = pd.concat([dataset, dataset_single_article], axis=0)

dataset.reset_index(drop=True, inplace=True)
dataset.to_csv('Articles.csv')
dataset

In [ ]:
dataset['Date'].unique()

### Stock market - WIG 20 historic price

In [ ]:
wig_20 = pd.read_csv('https://stooq.pl/q/d/l/?s=wig20&i=d')
wig_20.to_csv('WIG_20.csv')
wig_20